# Optimizing for CO2 storage from CO2 Emitters

## Setup

In [1]:
import numpy as np
import pandas as pd
from pulp import *
from geopy.distance import distance
from geopy.distance import great_circle
import random
import pickle

In [44]:
def facility_cleaning(fac_df, emiss_df):
    facilitiesB = fac_df.rename(inplace=False,
                                    columns={'Facility Name':'facility_name',
                                             'City':'city',
                                             'Primary NAICS Code':'naics_code',
                                             'Industry Type (subparts)':'industry_subparts',
                                             'Industry Type (sectors)':'industry_sectors'})

    emsCA2019loc = pd.merge(emiss_df, facilitiesB[['Facility Id',
                                                    'facility_name',
    #                                               'Facility Name2',
                                                   'city',
                                                   'Zip Code',
                                                   'Address',
                                                   'County',
                                                   'Latitude',
                                                   'Longitude',
    #                                               'Primary NAICS Code2',
                                                   'industry_subparts',
                                                   'industry_sectors']],
                            how='left', on='Facility Id')
    emsCA2019locagg = emsCA2019loc[['Facility Id','facility_name','Latitude','Longitude',
                                    'Unit CO2 emissions (non-biogenic)']].dropna().groupby(
                                        ['Facility Id','facility_name']).agg({'Latitude':'min',
                                                            'Longitude':'min',
                                                            'Unit CO2 emissions (non-biogenic)':'sum'})
    emsCA2019locagg.reset_index(inplace=True)
    emsCA2019locagg = emsCA2019locagg.rename(columns={'Facility Id':'facility_id',
                                                      'facility_name':'facility_name',
                                                      'Latitude':'lat',
                                                      'Longitude':'lon',
                                                      'Unit CO2 emissions (non-biogenic)':'annual_emissions'})
    emsCA2019locagg = emsCA2019locagg[emsCA2019locagg['annual_emissions'] > 0]
    facility_latlon = emsCA2019locagg[['lat', 'lon']].to_records(index=False)
    facility_ids = emsCA2019locagg['facility_id'].tolist()
    facility_names = emsCA2019locagg['facility_name'].tolist()
    supply_volume = emsCA2019locagg['annual_emissions'].tolist()
    supply_trucks = [i/25 for i in supply_volume]
    return facility_latlon, facility_ids, facility_names, supply_volume, supply_trucks


def get_distances(facility_latlon, pools_latlon):
    pools_distance_matrix = []
    for i in range(len(facility_latlon)):
        distances = []
        for j in range(len(pools_latlon)):
            dist = great_circle(facility_latlon[i], pools_latlon[j]).miles
            distances.append(dist)
        pools_distance_matrix.append(distances)
    return pools_distance_matrix


def get_complex_costs(distance_matrix, pool_trucks):
    complex_costs = []
    for i in range(len(distance_matrix)):
        new = []
        for j in range(len(distance_matrix[i])):
            opening_cost = 2000000/pool_trucks[j]
            # opening cost = $2,000,000/number of trucks for the pool
            # cost / truck = 2*$2*distance + opening cost
            cost = 4 * distance_matrix[i][j] + opening_cost
            new.append(cost)
        complex_costs.append(new)
    return complex_costs


def run_optimization(n_receivers, n_suppliers, supply_trucks, demand_trucks, costs, distances, facility_names, facility_latlon, pools_ids, pools_latlon, industry_str, year):
    print("Running optimization for {} industry and {} year timeframe".format(industry_str, year))
    supply_trucks = [i*year for i in supply_trucks]
    
    p_prob = LpProblem('Unbalaced_Transportation_Problem', LpMinimize)
    #costs = pools_distance_matrix_complexcost
    routes = [(i, j) for i in range(n_suppliers) for j in range(n_receivers)]

    x = LpVariable.dicts('X', routes, lowBound=0)
    p_prob += lpSum([x[i, j] * costs[i][j] for i in range(n_suppliers) for j in range(n_receivers)])

    for i in range(n_suppliers):
        p_prob += lpSum([x[i, j] for j in range(n_receivers)]) == supply_trucks[i]

    for j in range(n_receivers):
        p_prob += lpSum([x[i, j] for i in range(n_suppliers)]) <= demand_trucks[j]

    # Solving problem
    p_prob.solve()
    print("Problem successfully solved")
    
    volumes_moved = [i.varValue for i in p_prob.variables()]
    volume_matrix = []
    k=0
    while k < n_suppliers:
        vols = []
        for i in range(n_receivers):
            j = i + k*n_receivers
            vols.append(volumes_moved[j])
        volume_matrix.append(vols)
        k+=1
    print("Volume matrix created")
        
    results = []
    for i in range(n_suppliers):
        f_name = facility_names[i]
        f_loc = facility_latlon[i]
        for j in range(n_receivers):
            vol = volume_matrix[i][j]
            if vol > 0:
                w_id = pools_ids[j]
                w_loc = pools_latlon[j]
                dist = distances[i][j]
                cost = costs[i][j] * vol
                r = [f_name, f_loc, w_id, w_loc, dist, cost, vol, industry_str, year]
                results.append(r)
    results_df = pd.DataFrame(results, columns = ['facility_name', 'facility_location','pool_id','pool_location','distance','cost','n_trucks', 'industry', 'timeframe'])
    print("Results created")
    print()
    print("--------------------------------------------")
    
    return results_df


## Data import and cleaning

In [45]:
wells = pd.read_csv("data/wells/AllWells_20210915.csv")

# Oil & Gas, Dry Gas (no liquids), Gas and "Liquefied Gas"
wells_oilgas = wells[ (wells['WellType'] == 'OG')  |
                      (wells['WellType'] == 'DG')  |
                      (wells['WellType'] == 'GAS') |
                      (wells['WellType'] == 'LG') ]
wells_latlon = wells_oilgas[['Latitude', 'Longitude']].to_records(index=False)
wells_ids = wells_oilgas['API'].tolist()
pooled_wells = pd.read_csv("data/pool_volumes.csv")
pools_latlon = pooled_wells[['Latitude', 'Longitude']].to_records(index=False)
pools_ids = pooled_wells['pool_id'].tolist()
pools_volumes = pooled_wells['totalco2'].tolist()
pools_trucks = [i/25 for i in pools_volumes]
n_pools = len(pools_ids)

In [46]:
emissions = pd.read_csv(r'./data/Emissions by Unit and Fuel Type - csv/UNIT_DATA-Table 1.csv')
emsCA2019 = emissions[emissions['Reporting Year'] == 2019]
facilities = pd.read_excel(r'./data/ghgp_data_2019.xlsx',
                          sheet_name='Direct Emitters', skiprows=3)
# One Michegan emitter lists its CA headquarters. Retag it as Michigan
facilities.loc[facilities['Facility Id'] == 1000594, 'State'] = 'MI'

## Prepare data for optimization

In [47]:
facilities = facilities[(facilities.State == 'CA')]
facilities_cement = facilities[facilities['Cement Production'] > 0]
facilities_oil = facilities[facilities['Petroleum Refining'] > 0]
facilities_energy = facilities[facilities['Electricity Generation'] > 0]
facilities_steel = facilities[facilities['Iron and Steel Production'] > 0]

facility_latlon, facility_ids, facility_names, supply_volume, supply_trucks = facility_cleaning(facilities, emsCA2019)
n_facilities = len(facility_ids)

facility_c_latlon, facility_c_ids, facility_c_names, supply_c_volume, supply_c_trucks = facility_cleaning(facilities_cement, emsCA2019)
n_facilities_c = len(facility_c_ids)

facility_o_latlon, facility_o_ids, facility_o_names, supply_o_volume, supply_o_trucks = facility_cleaning(facilities_oil, emsCA2019)
n_facilities_o = len(facility_o_ids)

facility_e_latlon, facility_e_ids, facility_e_names, supply_e_volume, supply_e_trucks = facility_cleaning(facilities_energy, emsCA2019)
n_facilities_e = len(facility_e_ids)

facility_s_latlon, facility_s_ids, facility_s_names, supply_s_volume, supply_s_trucks = facility_cleaning(facilities_steel, emsCA2019)
n_facilities_s = len(facility_s_ids)

facility_total_latlon = np.concatenate((facility_c_latlon, facility_o_latlon, facility_e_latlon, facility_s_latlon))
facility_total_ids = np.concatenate((facility_c_ids, facility_o_ids, facility_e_ids, facility_s_ids))
facility_total_names = np.concatenate((facility_c_names, facility_o_names, facility_e_names, facility_s_names))
supply_volume_total = np.concatenate((supply_c_volume, supply_o_volume, supply_e_volume, supply_s_volume))
supply_trucks_total = np.concatenate((supply_c_trucks, supply_o_trucks, supply_e_trucks, supply_s_trucks))
n_facilities_total = len(facility_total_ids)

In [48]:
facilitiesB = facilities_cement.rename(inplace=False,
                                    columns={'Facility Name':'facility_name',
                                             'City':'city',
                                             'Primary NAICS Code':'naics_code',
                                             'Industry Type (subparts)':'industry_subparts',
                                             'Industry Type (sectors)':'industry_sectors'})
emsCA2019loc = pd.merge(emsCA2019, facilitiesB[['Facility Id',
                                                    'facility_name',
    #                                               'Facility Name2',
                                                   'city',
                                                   'Zip Code',
                                                   'Address',
                                                   'County',
                                                   'Latitude',
                                                   'Longitude',
    #                                               'Primary NAICS Code2',
                                                   'industry_subparts',
                                                   'industry_sectors']],
                            how='left', on='Facility Id')
emsCA2019locagg = emsCA2019loc[['Facility Id','facility_name','Latitude','Longitude',
                                    'Unit CO2 emissions (non-biogenic)']].dropna().groupby(
                                        ['Facility Id','facility_name']).agg({'Latitude':'min',
                                                            'Longitude':'min',
                                                            'Unit CO2 emissions (non-biogenic)':'sum'})
emsCA2019locagg.reset_index(inplace=True)
emsCA2019locagg = emsCA2019locagg.rename(columns={'Facility Id':'facility_id',
                                                      'facility_name':'facility_name',
                                                      'Latitude':'lat',
                                                      'Longitude':'lon',
                                                      'Unit CO2 emissions (non-biogenic)':'annual_emissions'})
emsCA2019locagg = emsCA2019locagg[emsCA2019locagg['annual_emissions'] > 0]
emsCA2019locagg

,facility_id,facility_name,lat,lon,annual_emissions
0,1002308,CEMEX Construction Materials Pacific LLC,34.622200,-117.100100,12834.3
1,1002431,HANSON PERMANENTE CEMENT,37.318100,-122.091000,464.9
2,1004612,LEHIGH SOUTHWEST CEMENT CO.,40.736900,-122.322300,1072.2
3,1005662,Mitsubishi Cement Corp Cushenbury Cement Plant,34.437557,-116.891034,421.7
4,1006642,NATIONAL CEMENT CO OF CALIFORNIA INC,34.819863,-118.748732,2476.8
5,1006842,CalPortland Company Mojave Plant,35.029298,-118.316236,712.0
6,1007927,CalPortland Company Oro Grande Plant,34.604500,-117.338200,270.5


In [49]:
distances_c = get_distances(facility_c_latlon, pools_latlon)
complex_costs_c = get_complex_costs(distances_c, pools_trucks)

distances_o = get_distances(facility_o_latlon, pools_latlon)
complex_costs_o = get_complex_costs(distances_o, pools_trucks)

distances_e = get_distances(facility_e_latlon, pools_latlon)
complex_costs_e = get_complex_costs(distances_e, pools_trucks)

distances_s = get_distances(facility_s_latlon, pools_latlon)
complex_costs_s = get_complex_costs(distances_s, pools_trucks)

distances_total = get_distances(facility_total_latlon, pools_latlon)
complex_costs_total = get_complex_costs(distances_total, pools_trucks)

In [50]:
industry_list = ['cement', 'oil_refining', 'steel', 'energy', 'total']
year_list = [1, 5, 10, 20, 30]

inputs_list = []
for i in year_list:
    cement = [n_facilities_c, supply_c_trucks, complex_costs_c, distances_c, facility_c_names, facility_c_latlon, 'cement', i]
    oil = [n_facilities_o, supply_o_trucks, complex_costs_o, distances_o, facility_o_names, facility_o_latlon, 'oil_refining', i]
    energy = [n_facilities_e, supply_e_trucks, complex_costs_e, distances_e, facility_e_names, facility_e_latlon, 'energy', i]
    steel = [n_facilities_s, supply_s_trucks, complex_costs_s, distances_s, facility_s_names, facility_s_latlon, 'steel', i]
    total = [n_facilities_total, supply_trucks_total, complex_costs_total, distances_total, facility_total_names, facility_total_latlon, 'total', i]
    
    inputs_list.append(cement)
    inputs_list.append(oil)
    inputs_list.append(steel)
    inputs_list.append(energy)
    inputs_list.append(total)

    

## Run optimization and write results

In [51]:
detailed_results = pd.DataFrame()

for i in inputs_list:
    results = run_optimization(n_pools,
                               i[0],
                               i[1],
                               pools_trucks,
                               i[2],
                               i[3],
                               i[4],
                               i[5],
                               pools_ids,
                               pools_latlon,
                               i[6],
                               i[7])
    detailed_results = detailed_results.append(results)


Running optimization for cement industry and 1 year timeframe
Problem successfully solved
Volume matrix created
Results created

--------------------------------------------
Running optimization for oil_refining industry and 1 year timeframe
Problem successfully solved
Volume matrix created
Results created

--------------------------------------------
Running optimization for steel industry and 1 year timeframe
Problem successfully solved
Volume matrix created
Results created

--------------------------------------------
Running optimization for energy industry and 1 year timeframe
Problem successfully solved
Volume matrix created
Results created

--------------------------------------------
Running optimization for total industry and 1 year timeframe
Problem successfully solved
Volume matrix created
Results created

--------------------------------------------
Running optimization for cement industry and 5 year timeframe
Problem successfully solved
Volume matrix created
Results create

In [52]:
# re calculate cost as 2 * ($2*miles)*(volume/25) + 2,000,000
detailed_results['volume'] = detailed_results['n_trucks']*25
detailed_results['cost_per_ton'] = detailed_results['cost']/detailed_results['volume']
detailed_results.sort_values(by=['cost_per_ton'])

,facility_name,facility_location,pool_id,pool_location,distance,cost,n_trucks,industry,timeframe,volume,cost_per_ton
53,ULTRAMAR INC WILMINGTON REFINERY,"[33.7783, -118.2351]",Wilmington:Fault Block 90:Ranger,"[33.73989, -118.139243]",6.112491,7.314882e+05,24695.3510,total,30,6.173838e+05,1.184819
172,La Paloma Generating Plant,"[35.2956, -119.5919]",McKittrick:Northeast:Point of Rocks,"[35.322624000000005, -119.6388595]",3.239887,7.535933e+05,13648.5690,energy,30,3.412142e+05,2.208563
77,"Delta Energy Center, LLC","[38.0167, -121.8439]","Brentwood, East, Gas::Third Massive","[37.940883, -121.678394]",10.425451,1.082165e+06,18405.3060,energy,20,4.601327e+05,2.351854
161,"Delta Energy Center, LLC","[38.0167, -121.8439]","Brentwood, East, Gas::Third Massive","[37.940883, -121.678394]",10.425451,1.082165e+06,18405.3060,energy,30,4.601327e+05,2.351854
270,La Paloma Generating Plant,"[35.2956, -119.5919]",Belgian Anticline:Main:npool,"[35.2936305, -119.656802]",3.662579,7.662875e+05,11450.9870,total,30,2.862747e+05,2.676756
...,...,...,...,...,...,...,...,...,...,...,...
8,ULTRAMAR INC WILMINGTON REFINERY,"[33.7783, -118.2351]","Belridge, North::","[35.53990555, -119.7747803]",149.900040,2.762148e+12,122697.6000,oil_refining,5,3.067440e+06,900473.325946
31,ULTRAMAR INC WILMINGTON REFINERY,"[33.7783, -118.2351]","Belridge, North::","[35.53990555, -119.7747803]",149.900040,2.762148e+12,122697.6000,total,30,3.067440e+06,900473.325946
27,ULTRAMAR INC WILMINGTON REFINERY,"[33.7783, -118.2351]","Belridge, North::","[35.53990555, -119.7747803]",149.900040,2.762148e+12,122697.6000,oil_refining,10,3.067440e+06,900473.325946
26,ULTRAMAR INC WILMINGTON REFINERY,"[33.7783, -118.2351]","Belridge, North::","[35.53990555, -119.7747803]",149.900040,4.317865e+11,19180.4250,total,20,4.795106e+05,900473.325946


In [53]:
summary_results = detailed_results.groupby(by=['industry','timeframe']).agg(
    volume_transferred=('volume', 'sum'),
    total_cost=('cost', 'sum'),
    total_distance=('distance', 'sum')  
)
summary_results['cost_per_ton'] = summary_results['total_cost']/summary_results['volume_transferred']
summary_results



volume_transferred    total_cost  total_distance  \
industry     timeframe                                                     
cement       1                1.825240e+04  1.017482e+07     1343.970661   
             5                9.126200e+04  5.087410e+07     1343.970661   
             10               1.825240e+05  1.017482e+08     1343.970661   
             20               3.650480e+05  2.034964e+08     1343.970661   
             30               5.475720e+05  3.052446e+08     1343.970661   
energy       1                2.795162e+07  1.035760e+12    16156.576702   
             5                1.397581e+08  5.199723e+12    18861.650510   
             10               2.795162e+08  6.546392e+12    24872.893488   
             20               5.590323e+08  7.869839e+12    32792.606566   
             30               8.385485e+08  8.186786e+12    51978.944864   
oil_refining 1                1.735662e+07  6.898195e+11     2697.253780   
             5                8.678310e+07  7.739281e+12     4423.195980   
             10               1.735662e+08  7.769294e+12    10510.463684   
             20               3.471324e+08  7.813952e+12    13639.171865   
             30               5.206986e+08  7.983895e+12    21582.942931   
steel        1                3.330860e+04  1.762145e+07       45.538919   
             5                1.665430e+05  8.810723e+07       45.538919   
             10               3.330860e+05  1.762145e+08       45.538919   
             20               6.661720e+05  3.524289e+08       45.538919   
             30               9.992580e+05  5.286434e+08       45.538919   
total        1                4.535980e+07  1.725609e+12    19506.935523   
             5                2.267990e+08  7.787447e+12    27666.159057   
             10               4.535980e+08  7.835590e+12    36430.493633   
             20               9.071959e+08  8.204622e+12    69649.856358   
             30               1.360794e+09  8.329992e+12    81397.583660   

                        cost_per_ton  
industry     timeframe                
cement       1            557.451028  
             5            557.451028  
             10           557.451028  
             20           557.451028  
             30           557.451028  
energy       1          37055.444215  
             5          37205.171085  
             10         23420.440685  
             20         14077.610736  
             30          9763.044395  
oil_refining 1          39743.884076  
             5          89179.586206  
             10         44762.714449  
             20         22510.004891  
             30         15333.044399  
steel        1            529.035923  
             5            529.035923  
             10           529.035923  
             20           529.035923  
             30           529.035923  
total        1          38042.702197  
             5          34336.339180  
             10         17274.305142  
             20          9043.935830  
             30          6121.420545

In [54]:
detailed_results['volume'] = detailed_results['volume']/1000
detailed_results['cost'] = detailed_results['cost']/1000000

In [55]:
summary_results['volume_transferred'] = summary_results['volume_transferred']/1000
summary_results['total_cost'] = summary_results['total_cost']/1000000

In [56]:
detailed_results.sort_values(by=['cost_per_ton'])

,facility_name,facility_location,pool_id,pool_location,distance,cost,n_trucks,industry,timeframe,volume,cost_per_ton
53,ULTRAMAR INC WILMINGTON REFINERY,"[33.7783, -118.2351]",Wilmington:Fault Block 90:Ranger,"[33.73989, -118.139243]",6.112491,7.314882e-01,24695.3510,total,30,617.383775,1.184819
172,La Paloma Generating Plant,"[35.2956, -119.5919]",McKittrick:Northeast:Point of Rocks,"[35.322624000000005, -119.6388595]",3.239887,7.535933e-01,13648.5690,energy,30,341.214225,2.208563
77,"Delta Energy Center, LLC","[38.0167, -121.8439]","Brentwood, East, Gas::Third Massive","[37.940883, -121.678394]",10.425451,1.082165e+00,18405.3060,energy,20,460.132650,2.351854
161,"Delta Energy Center, LLC","[38.0167, -121.8439]","Brentwood, East, Gas::Third Massive","[37.940883, -121.678394]",10.425451,1.082165e+00,18405.3060,energy,30,460.132650,2.351854
270,La Paloma Generating Plant,"[35.2956, -119.5919]",Belgian Anticline:Main:npool,"[35.2936305, -119.656802]",3.662579,7.662875e-01,11450.9870,total,30,286.274675,2.676756
...,...,...,...,...,...,...,...,...,...,...,...
8,ULTRAMAR INC WILMINGTON REFINERY,"[33.7783, -118.2351]","Belridge, North::","[35.53990555, -119.7747803]",149.900040,2.762148e+06,122697.6000,oil_refining,5,3067.440000,900473.325946
31,ULTRAMAR INC WILMINGTON REFINERY,"[33.7783, -118.2351]","Belridge, North::","[35.53990555, -119.7747803]",149.900040,2.762148e+06,122697.6000,total,30,3067.440000,900473.325946
27,ULTRAMAR INC WILMINGTON REFINERY,"[33.7783, -118.2351]","Belridge, North::","[35.53990555, -119.7747803]",149.900040,2.762148e+06,122697.6000,oil_refining,10,3067.440000,900473.325946
26,ULTRAMAR INC WILMINGTON REFINERY,"[33.7783, -118.2351]","Belridge, North::","[35.53990555, -119.7747803]",149.900040,4.317865e+05,19180.4250,total,20,479.510625,900473.325946


In [62]:
list_results = detailed_results.values.tolist()
file_name = 'data/results/full_detailed_results.pkl'
open_file = open(file_name, "wb")
pickle.dump(list_results, open_file)
open_file.close()

html_results = detailed_results.to_html()
html_file = open("data/results/full_detailed_results.html", "w")
html_file.write(html_results)
html_file.close()

In [58]:
total_pool_volume = sum(pools_volumes)/1000

summary_results['percent_pool_volume_used'] = summary_results['volume_transferred'] / total_pool_volume
summary_results

volume_transferred    total_cost  total_distance  \
industry     timeframe                                                     
cement       1                1.825240e+01  1.017482e+01     1343.970661   
             5                9.126200e+01  5.087410e+01     1343.970661   
             10               1.825240e+02  1.017482e+02     1343.970661   
             20               3.650480e+02  2.034964e+02     1343.970661   
             30               5.475720e+02  3.052446e+02     1343.970661   
energy       1                2.795162e+04  1.035760e+06    16156.576702   
             5                1.397581e+05  5.199723e+06    18861.650510   
             10               2.795162e+05  6.546392e+06    24872.893488   
             20               5.590323e+05  7.869839e+06    32792.606566   
             30               8.385485e+05  8.186786e+06    51978.944864   
oil_refining 1                1.735662e+04  6.898195e+05     2697.253780   
             5                8.678310e+04  7.739281e+06     4423.195980   
             10               1.735662e+05  7.769294e+06    10510.463684   
             20               3.471324e+05  7.813952e+06    13639.171865   
             30               5.206986e+05  7.983895e+06    21582.942931   
steel        1                3.330860e+01  1.762145e+01       45.538919   
             5                1.665430e+02  8.810723e+01       45.538919   
             10               3.330860e+02  1.762145e+02       45.538919   
             20               6.661720e+02  3.524289e+02       45.538919   
             30               9.992580e+02  5.286434e+02       45.538919   
total        1                4.535980e+04  1.725609e+06    19506.935523   
             5                2.267990e+05  7.787447e+06    27666.159057   
             10               4.535980e+05  7.835590e+06    36430.493633   
             20               9.071959e+05  8.204622e+06    69649.856358   
             30               1.360794e+06  8.329992e+06    81397.583660   

                        cost_per_ton  percent_pool_volume_used  
industry     timeframe                                          
cement       1            557.451028                  0.000011  
             5            557.451028                  0.000053  
             10           557.451028                  0.000105  
             20           557.451028                  0.000210  
             30           557.451028                  0.000315  
energy       1          37055.444215                  0.016088  
             5          37205.171085                  0.080441  
             10         23420.440685                  0.160881  
             20         14077.610736                  0.321762  
             30          9763.044395                  0.482643  
oil_refining 1          39743.884076                  0.009990  
             5          89179.586206                  0.049950  
             10         44762.714449                  0.099899  
             20         22510.004891                  0.199799  
             30         15333.044399                  0.299698  
steel        1            529.035923                  0.000019  
             5            529.035923                  0.000096  
             10           529.035923                  0.000192  
             20           529.035923                  0.000383  
             30           529.035923                  0.000575  
total        1          38042.702197                  0.026108  
             5          34336.339180                  0.130539  
             10         17274.305142                  0.261077  
             20          9043.935830                  0.522155  
             30          6121.420545                  0.783232

In [60]:
list_summary_results = summary_results.values.tolist()
file_name = 'data/results/summary_results.pkl'
open_file = open(file_name, "wb")
pickle.dump(list_summary_results, open_file)
open_file.close()

html_summary_results = summary_results.to_html()
html_file = open("data/results/summary_results.html", "w")
html_file.write(html_summary_results)
html_file.close()